In [ ]:
import os
os.kill(os.getpid(), 9)  # This will force a Colab restart


Download and read dataset.
   
It contains 3 tables - 'Users.csv', 'Ratings.csv', 'Books.csv'.

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3


In [4]:
 path = '/home/jovyan/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3'

In [5]:
import os

files = os.listdir(path)
print(files)

['DeepRec.png', 'Users.csv', 'Ratings.csv', 'Books.csv', 'classicRec.png', 'recsys_taxonomy2.png']


In [6]:

import pandas as pd

books = pd.read_csv(os.path.join(path, 'Books.csv'))
books.head(5)


/tmp/ipykernel_85/66192168.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(os.path.join(path, 'Books.csv'))


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
users = pd.read_csv(os.path.join(path, 'Users.csv'))
users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
ratings = pd.read_csv(os.path.join(path, 'Ratings.csv'))
ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [29]:
ratings[ratings['ISBN']=='0195153448']

,User-ID,ISBN,Book-Rating,user_index,book_index
9561,2,0195153448,0,850,8967


In [35]:
ratings[ratings['User-ID']==276729]

,User-ID,ISBN,Book-Rating,user_index,book_index
3,276729,052165615X,3,3,3
4,276729,0521795028,6,3,4


Below is implemented a Collaborative Filtering-based book recommendation system using Alternating Least Squares (ALS), a matrix factorization technique provided by the implicit library. The system processes a dataset of book ratings, trains an ALS model, and recommends books to users based on their past ratings.

The very first thing is to extract unique user IDs and unique book ISBNs from the ratings.
Create mapping dictionaries to be able to work with sparse matrices - (user_map and book_map) that map:
-- Each User-ID to a numeric index.
-- Each ISBN to a numeric index.
Add mapped data to the ratings.


In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit
import os



# Create user and book mappings
user_ids = ratings['User-ID'].unique()
book_isbns = ratings['ISBN'].unique()

user_map = {user_id: index for index, user_id in enumerate(user_ids)}
book_map = {isbn: int(index) for index, isbn in enumerate(book_isbns)}

ratings['user_index'] = ratings['User-ID'].map(user_map)
ratings['book_index'] = ratings['ISBN'].map(book_map)


Below is constructed a Coordinate (COO) sparse matrix:
-- Rows represent users.
-- Columns represent books.
-- Values represent ratings.
The matrix is converted to Compressed Sparse Row (CSR) format for efficient mathematical operations.

Below is used Collaborative Filtering (CF) using Alternating Least Squares (ALS)

Collaborative Filtering (CF): Recommends books based on user-item interactions.

Matrix Factorization: ALS decomposes the user-book rating matrix into two lower-dimensional matrices:
-- User matrix (U)
-- Item matrix (V)

The dot product of these two matrices approximates the original ratings.

In [ ]:
# Create sparse matrix
sparse_matrix = sparse.coo_matrix((ratings['Book-Rating'], (ratings['user_index'], ratings['book_index'])), 
                                 shape=(len(user_ids), len(book_isbns))).tocsr()

# Train the ALS model
als_model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(sparse_matrix)

# Create a reverse book map
book_index_to_isbn = {index: isbn for isbn, index in book_map.items()}


Function below returns book recommendations using the ALS model. It returns the top n=5 recommended book indices.
It converts book indices back to ISBNs using book_index_to_isbn and loads book details from Books.csv and filter based on the recommended ISBNs.


Get book recommendations using the ALS model:
als_model.recommend(user_idx, user_items_csr, N=n) returns the top n recommended book indices.
Convert book indices back to ISBNs using book_index_to_isbn.
Load book details from Books.csv and filter based on the recommended ISBNs.


In [ ]:
def recommend_books_cf(user_id, n=5):
    """Recommends books for a user using ALS collaborative filtering."""
    if user_id not in user_map:
        return []

    user_idx = user_map[user_id]

    # Extract the user's row from the transposed matrix
    user_items_row = sparse_matrix.T[user_idx]
    user_items_csr = sparse.csr_matrix(user_items_row)

    # Get recommended book indices
    recommended_results = als_model.recommend(user_idx, user_items_csr, N=n)

    # Extract the item indices
    recommended_indices = recommended_results[0]  # Corrected line

    # Convert book indices back to ISBNs using the reverse map
    recommended_books = [book_index_to_isbn.get(i) for i in recommended_indices if book_index_to_isbn.get(i) is not None]

    # Fetch book details (assuming 'Books.csv' is also in the same directory)
    books = pd.read_csv(os.path.join(path, 'Books.csv'))  # Load book details
    return books[books["ISBN"].isin(recommended_books)][["ISBN", "Book-Title", "Book-Author"]].to_dict(orient="records")



In [36]:
#  Usage
recommendations_cf = recommend_books_cf(276729, 5)
for book in recommendations_cf:
    print(f"{book['Book-Title']} by {book['Book-Author']} (ISBN: {book['ISBN']})")

print(ratings.dtypes)

100%|██████████| 20/20 [01:13<00:00,  3.68s/it]


Wild Animus by Rich Shapero (ISBN: 0971880107)
Little Altars Everywhere: A Novel by Rebecca Wells (ISBN: 0060976845)
Fahrenheit 451 by RAY BRADBURY (ISBN: 0345342968)
The Hobbit : The Enchanting Prelude to The Lord of the Rings by J.R.R. TOLKIEN (ISBN: 0345339681)
The Pilot's Wife : A Novel by Anita Shreve (ISBN: 0316601950)
User-ID         int64
ISBN           object
Book-Rating     int64
user_index      int64
book_index      int64
dtype: object


/tmp/ipykernel_85/3301709481.py:55: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(os.path.join(path, 'Books.csv'))  # Load book details


In [25]:
book_map = {isbn: index for index, isbn in enumerate(book_isbns)}
print("Book Map Keys:", list(book_map.keys())[:5])  # Print first 5 keys
print("Type of Book Map Keys:", type(list(book_map.keys())[0]))
print("Book Map Values:", list(book_map.values())[:5]) #print first 5 values.
print("Type of Book Map Values:", type(list(book_map.values())[0]))

Book Map Keys: ['034545104X', '0155061224', '0446520802', '052165615X', '0521795028']
Type of Book Map Keys: <class 'str'>
Book Map Values: [0, 1, 2, 3, 4]
Type of Book Map Values: <class 'int'>
